# 📘 SQL-Based Chatbot using Groq API
## This notebook demonstrates how to build a SQL-based answering chatbot using the Groq API.
## The chatbot interprets natural language questions, converts them into SQL queries, executes them against a local SQLite database and returns conversational answers. It leverages the power of Groq's LLaMA-3.1 models for fast and reliable responses.
## This is useful for turning structured database data into accessible, natural language insights.

# 🔧 Installing Required Libraries
## We begin by installing the required Python libraries: `groq` and `sqlite3` (built-in in Python).

In [ ]:
!pip install -q groq


# 🔐 Set Groq API Key
## Directly paste your Groq API key below.
## If you don't have one, get it from the [Groq Console](https://console.groq.com/keys).
## If you don't already have an account with GroqCloud, you can create one for free.
## ⚠️ NOTE: Never share this notebook publicly with your API key still in it.


In [ ]:
GROQ_API_KEY = "your-groq-api-key-here"  # 🔁 Replace this with your actual API key

from groq import Groq

# Initialize the Groq client
client = Groq(api_key=GROQ_API_KEY)


# 🗃️ Create a Sample SQLite Database
## We'll create an in-memory SQLite database with a `students` table.

In [ ]:
import sqlite3

conn = sqlite3.connect(":memory:")  # Temporary in-memory database
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    grade TEXT
)
""")

# Insert some sample rows
students = [
    (1, "Alice", 20, "A"),
    (2, "Bob", 21, "B"),
    (3, "Charlie", 19, "A"),
    (4, "David", 22, "C"),
    (5, "Eva", 20, "B")
]
cursor.executemany("INSERT INTO students VALUES (?, ?, ?, ?)", students)
conn.commit()


# 📑 Define a Function to Convert Natural Language to SQL
## This function calls the Groq LLM to turn user questions into valid SQL queries.


In [ ]:
def generate_sql_query(question: str, table_schema: str) -> str:
    system_prompt = f"""
You are a data analyst assistant.
Convert the user's natural language question into a syntactically correct SQLite query using the schema below.
Respond ONLY with the SQL query (no comments, no explanation).

Schema:
{table_schema}
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    response = client.chat.completions.create(
        model="llama-3-8b-instruct",
        messages=messages,
        temperature=0.2,
        max_tokens=150,
    )

    return response.choices[0].message.content.strip()


# 🔍 Extract Schema from the Database
## This helper function dynamically fetches table column information to inform the model.


In [ ]:
def get_table_schema(connection, table_name="students"):
    cursor = connection.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    formatted_schema = f"Table: {table_name}\nColumns:\n"
    for col in schema:
        formatted_schema += f"  - {col[1]} ({col[2]})\n"
    return formatted_schema

schema = get_table_schema(conn)
print(schema)

# 🧪 Try the SQL Generator
## We test our `generate_sql_query` function with a simple question.


In [ ]:
question = "Which students have grade A?"
sql_query = generate_sql_query(question, schema)
print("Generated SQL:\n", sql_query)

# ▶️ Execute SQL on SQLite
## This function runs the generated SQL query and returns the results.


In [ ]:
def execute_sql_query(connection, query: str):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    except sqlite3.Error as e:
        return f"SQL error: {e}"

results = execute_sql_query(conn, sql_query)
print("Results:\n", results)

# 🤖 Full Chatbot Pipeline
## This wrapper puts everything together: schema → prompt → SQL → result.

In [ ]:
def chatbot_answer(question: str) -> str:
    schema = get_table_schema(conn)
    sql = generate_sql_query(question, schema)
    result = execute_sql_query(conn, sql)
    return f"Generated SQL:\n{sql}\n\nAnswer:\n{result}"

# Try it out
print(chatbot_answer("List the names of students who are 20 years old."))

# 📙 Conclusion
## In this notebook, we built a simple yet powerful SQL-aware chatbot using Groq's LLaMA models and SQLite.

## ✅ Highlights:
## - We used LLMs to translate natural language into SQL.
## - Extracted schema dynamically and executed generated queries.
## - Used Groq's blazing-fast inference for near-instant interaction.

## 🚀 Next Steps:
## - Connect this to real-world databases (PostgreSQL, MySQL).
## - Extend support to multiple tables and JOINs.
## - Add a front-end using Streamlit or Gradio.